In [1]:
import cv2
import numpy as np
import math
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import io


In [2]:
global img_3
img_3 = np.zeros([300,300,3],dtype=np.uint8)
img_3.fill(255)
(h, w) = img_3.shape[:2]
center = (w / 2, h / 2)

In [3]:
import cv2
import math
import numpy as np
import sys

def apply_mask(matrix, mask, fill_value):
    masked = np.ma.array(matrix, mask=mask, fill_value=fill_value)
    return masked.filled()

def apply_threshold(matrix, low_value, high_value):
    low_mask = matrix < low_value
    matrix = apply_mask(matrix, low_mask, low_value)

    high_mask = matrix > high_value
    matrix = apply_mask(matrix, high_mask, high_value)

    return matrix

def simplest_cb(img, percent):
    assert img.shape[2] == 3
    assert percent > 0 and percent < 100

    half_percent = percent / 200.0

    channels = cv2.split(img)

    out_channels = []
    for channel in channels:
        assert len(channel.shape) == 2
        # find the low and high precentile values (based on the input percentile)
        height, width = channel.shape
        vec_size = width * height
        flat = channel.reshape(vec_size)

        assert len(flat.shape) == 1

        flat = np.sort(flat)

        n_cols = flat.shape[0]

        low_val  = flat[math.floor(n_cols * half_percent)]
        high_val = flat[math.ceil( n_cols * (1.0 - half_percent))]

        print("Lowval: ", low_val)
        print("Highval: ", high_val)

        # saturate below the low percentile and above the high percentile
        thresholded = apply_threshold(channel, low_val, high_val)
        # scale the channel
        normalized = cv2.normalize(thresholded, thresholded.copy(), 0, 255, cv2.NORM_MINMAX)
        out_channels.append(normalized)

    return cv2.merge(out_channels)

In [4]:
def compute_hist(img):
    hist = np.zeros((256,), np.uint8)
    h, w = img.shape[:2]
    for i in range(h):
        for j in range(w):
            hist[img[i][j]] += 1
    return hist
def equal_hist(hist):
    cumulator = np.zeros_like(hist, np.float64)
    for i in range(len(cumulator)):
        cumulator[i] = hist[:i].sum()
    #print(cumulator)
    new_hist = (cumulator - cumulator.min())/(cumulator.max() - cumulator.min()) * 255
    #new_hist = np.uint8(new_hist)
    return new_hist

In [5]:
while(True):
    cv2.imshow('Window', img_3)
    k = cv2.waitKey(0) 
    if k == ord('b'): #color balance
        img_3 = simplest_cb(img_3, 98)
    if k == ord('l'): # load
        image_location = input('Paste the image location:')
        img_3 = cv2.imread(image_location)
        cv2.imshow('Window', img_3)
    if k == ord('c'): # histogram equalization
        hist = compute_hist(img_3[:,:,0])
        new_hist = equal_hist(hist)
        img_yuv = cv2.cvtColor(img_3, cv2.COLOR_BGR2YUV)
        img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
        img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
        img_3 = img_output
    if k == ord('v'): #another color balanced
        B = img_3[:,:,0].max()
        G = img_3[:,:,1].max()
        R = img_3[:,:,2].max()
        CB = np.array([[255/B,0,0],[0,255/G,0],[0,0,255/R]])
        for row in range(img_3.shape[0]):
            for col in range(img_3.shape[1]):
                img_3[row,col] = np.dot(CB,img_3[row,col])
    if k == ord('h'): # show and hide histograms
        histr_1 = cv2.calcHist([img_3],[0],None,[256],[0,256])
        plt.plot(histr_1, c = 'k',label = 'histequal')
        plt.savefig('histogram.png')
        his_img = cv2.imread('histogram.png')
        plt.clf()
        cv2.imshow('Blended Image',his_img)
        cv2.waitKey(0)
    if k == ord('d'): #medium blur
        img_3 = cv2.medianBlur(img_3,3)
    if k == ord('n'): # box blur
        img_3 = cv2.blur(img_3,(3,3))
    if k == ord('g'): # gaussian blur
        img_3 = cv2.GaussianBlur(img_3,(3,3), cv2.BORDER_DEFAULT)
    if k == ord('x'): # clear image
        img_3.fill(255)
    if (k == 113): # quit
        break
cv2.destroyAllWindows()